In [23]:
import cfbd
import requests
import json
from itertools import islice
import time
from cfbd.rest import ApiException
from pprint import pprint
import sys, subprocess
import networkx as nx
from config import API_KEY

In [37]:
# access api

configuration = cfbd.Configuration(
    access_token = API_KEY
)


# Enter a context with an instance of the API client
with cfbd.ApiClient(configuration) as api_client:
    # Create an instance of the API class
    api_instance = cfbd.PlayersApi(api_client)

api_response_2023 = api_instance.get_transfer_portal(year=2023)
api_response_2022 = api_instance.get_transfer_portal(year=2022)
api_response_2021 = api_instance.get_transfer_portal(year=2021)
api_response_2020 = api_instance.get_transfer_portal(year=2020)
api_response_2019 = api_instance.get_transfer_portal(year=2019)


In [42]:
# convert to graphml

data = api_response_2019
filename = "transfer_portal_2019.graphml"

G = nx.DiGraph()

# Add nodes and edges to the graph G based on data
# Nodes: school names (origin and destination)
# Edges: a directed edge origin -> destination per player; edge attributes aggregate players
for t in data:
    origin = t.origin.strip() if getattr(t, 'origin', None) else None
    dest = t.destination.strip() if getattr(t, 'destination', None) else None

    # add nodes if present
    if origin:
        G.add_node(origin)
    if dest:
        G.add_node(dest)

    # only create edges when both origin and destination exist
    if origin and dest:
        player = f"{getattr(t, 'first_name', '')} {getattr(t, 'last_name', '')}".strip()
        pos = getattr(t, 'position', None)
        date = getattr(t, 'transfer_date', None)
        rating = getattr(t, 'rating', None)
        stars = getattr(t, 'stars', None)
        eligibility = getattr(t, 'eligibility', None)

        if G.has_edge(origin, dest):
            edge = G[origin][dest]
            edge.setdefault('players', []).append(player)
            edge.setdefault('positions', []).append(pos)
            edge.setdefault('dates', []).append(date)
            edge.setdefault('ratings', []).append(rating)
            edge.setdefault('stars', []).append(stars)
            edge.setdefault('eligibility', []).append(str(eligibility))
            edge['weight'] = edge.get('weight', 1) + 1
        else:
            G.add_edge(origin, dest, players=[player], positions=[pos], date=[date], ratings=[rating], stars=[stars], eligibility=[str(eligibility)], weight=1)


# Serialize any list-valued (or None) edge attributes to strings for GraphML compatibility
for u, v, attrs in G.edges(data=True):
    for k in list(attrs.keys()):
        val = attrs[k]
        if isinstance(val, list):
            # join list elements into a single string; convert None -> empty string
            attrs[k] = ' | '.join(['' if x is None else str(x) for x in val])
        elif val is None:
            attrs[k] = ''

# write graphml
nx.write_graphml(G, filename)
print(f"Wrote {len(G.nodes())} nodes and {len(G.edges())} edges to {filename}]")

Wrote 0 nodes and 0 edges to transfer_portal_2019.graphml]
